In [1]:
import pandas as pd

xtrain_raw = pd.read_csv('X_train.csv', index_col=0)
xtest_raw = pd.read_csv('X_test.csv', index_col=0)
ytrain_raw = pd.read_csv('y_train.csv', index_col=0)

xtrain_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27934 entries, 29083 to 15795
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      27934 non-null  object 
 1   has_test                  27934 non-null  bool   
 2   response_letter_required  27934 non-null  bool   
 3   salary_from               23902 non-null  float64
 4   salary_currency           27934 non-null  object 
 5   salary_gross              27786 non-null  object 
 6   published_at              27934 non-null  object 
 7   created_at                27934 non-null  object 
 8   employer_name             27934 non-null  object 
 9   description               27933 non-null  object 
 10  area_id                   27934 non-null  int64  
 11  area_name                 27934 non-null  object 
dtypes: bool(2), float64(1), int64(1), object(8)
memory usage: 2.4+ MB


In [67]:
reg='плата'
print(xtrain_raw['description'].str.contains(reg, regex=True).sum(), xtest_raw['description'].str.contains(reg, regex=True).sum())

12783 4313


In [37]:
xtrain_raw['salary_from'].describe()

count    2.390200e+04
mean     6.417946e+04
std      5.553079e+04
min      1.000000e+00
25%      3.350000e+04
50%      5.000000e+04
75%      8.000000e+04
max      3.400000e+06
Name: salary_from, dtype: float64

In [68]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer 
import scipy.sparse as sparse
from scipy.sparse import hstack, csr_matrix
import math

# xtr = xtrain_raw.drop(['Id'], axis=1)
# xtst = xtest_raw.drop(['Id'], axis=1)
xtr = xtrain_raw.copy()
xtst = xtest_raw.copy()


xtr.drop('created_at', axis=1, inplace=True)
xtst.drop('created_at', axis=1, inplace=True)

xtr['published_at'].ffill(inplace=True)
xtr['year'] = pd.to_datetime(xtr['published_at']).dt.year
xtr['month'] = pd.to_datetime(xtr['published_at']).dt.month
xtr.drop('published_at', axis=1, inplace=True)
xtst['published_at'].ffill(inplace=True)
xtst['year'] = pd.to_datetime(xtst['published_at']).dt.year
xtst['month'] = pd.to_datetime(xtst['published_at']).dt.month
xtst.drop('published_at', axis=1, inplace=True)

xtr['emp_length'] = xtr['employer_name'].str.len()
xtr['emp_length'].fillna(0.0, inplace=True)
xtr.drop('employer_name', axis=1, inplace=True)
xtst['emp_length'] = xtst['employer_name'].str.len()
xtst['emp_length'].fillna(0.0, inplace=True)
xtst.drop('employer_name', axis=1, inplace=True)

desclabels = ['hasremote','hashigh','hassocial','hasfree','hastk','haseng',
              'haslearn','hasinter','hasfivetwo','hastwotwo','hasflex','haspart',
              'hasprem', 'hassub', 'hasgrow', 'home', 'office',
             'hasport','hasfriend','hasfuture','hasbig','hasexpr',
             'hasneg','hasimpr','hasself','hascreat','hasinterest',
             'haschall','hascool','hascorp','haspunct','hasamb',
             'haswhite','hasyoung','hasstable','hasclient','haspay']
descregexps = ['удален|Удален', 'высш|Высш', 'страх|Страх|полис|ДМС', 'еспл', 'ТК РФ', 'англ|Англ|english|English',
               'обучение|Обучение','еждународ','5/2','2/2','ибкий', 'еполн', 
               'реми', 'метро', 'рост|рост', 'дом|Дом', 'офис|Офис',
               'ортфолио','дружн|Дружн','ерспек', 'круп|Круп','опыт|Опыт',
               'обсужд','азвити','амостоя','тветстве','нтерес'
               'спыта', 'Крут|крут','орпорати','унктуаль','амбиц|Амбиц',
               'елая','олод','табиль','лиент','плата']

for l, r in zip(desclabels, descregexps):
    xtr[l] = np.where(xtr['description'].str.contains(r, regex=True), True, False)
    xtst[l] = np.where(xtst['description'].str.contains(r, regex=True), True, False)

xtr['description_length'] = xtr['description'].str.len()/1000.0
xtr['description_length'].fillna(0.0, inplace=True)
xtst['description_length'] = xtst['description'].str.len()/1000.0
xtst['description_length'].fillna(0.0, inplace=True)

xtr['ismoscow'] = np.where(xtr['area_id'] == 1, True, False)
xtr.drop(['area_id', 'area_name'], axis=1, inplace = True)
xtst['ismoscow'] = np.where(xtst['area_id'] == 1, True, False)
xtst.drop(['area_id', 'area_name'], axis=1, inplace = True)

#mrot
xtr['salary_from'].fillna(15000.0, inplace=True)
xtst['salary_from'].fillna(15000.0, inplace=True)

# actually already normalized
def salcal(row):
    res = row['salary_from']
    if row['salary_currency'] != 'RUR':
        res = res/80.0
    if row['salary_gross'] == True:
        res = res*0.87
    return res - 15.000

xtr['accurate_salary'] = xtr.apply(salcal, axis=1)
xtr.drop(['salary_currency', 'salary_from', 'salary_gross'], axis=1, inplace=True)
xtst['accurate_salary'] = xtst.apply(salcal, axis=1)
xtst.drop(['salary_currency', 'salary_from', 'salary_gross'], axis=1, inplace=True)

xtr.shape, xtst.shape

xtr['dev'] = np.where(xtr['name'].str.contains('evelop|рограммист|азработчик', regex=True), True, False)
xtr['man'] = np.where(xtr['name'].str.contains('anag|енеджер', regex=True), True, False)
xtr['anal'] = np.where(xtr['name'].str.contains('nal|налит', regex=True), True, False)
xtr['adm'] = np.where(xtr['name'].str.contains('дмин', regex=True), True, False)
xtr['ruk'] = np.where(xtr['name'].str.contains('уковод|иректор|прав', regex=True), True, False)
xtr['oper'] = np.where(xtr['name'].str.contains('пер', regex=True), True, False)
xtr['prod'] = np.where(xtr['name'].str.contains('родав', regex=True), True, False)
xtr['engi'] = np.where(xtr['name'].str.contains('нжене', regex=True), True, False)
xtr['hr'] = np.where(xtr['name'].str.contains('hr|HR|екрут|ecruit', regex=True), True, False)
xtr['cln'] = np.where(xtr['name'].str.contains('борщ|линин', regex=True), True, False)
xtr['law'] = np.where(xtr['name'].str.contains('юр|Юр', regex=True), True, False)
xtr['prep'] = np.where(xtr['name'].str.contains('епод|читель', regex=True), True, False)
xtr['pov'] = np.where(xtr['name'].str.contains('овар|екарь', regex=True), True, False)
xtr['buh'] = np.where(xtr['name'].str.contains('ухгал', regex=True), True, False)

xtr.drop('name', axis=1, inplace=True)

xtst['dev'] = np.where(xtst['name'].str.contains('evelop|рограммист|азработчик', regex=True), True, False)
xtst['man'] = np.where(xtst['name'].str.contains('anag|енеджер', regex=True), True, False)
xtst['anal'] = np.where(xtst['name'].str.contains('nal|налит', regex=True), True, False)
xtst['adm'] = np.where(xtst['name'].str.contains('дмин', regex=True), True, False)
xtst['ruk'] = np.where(xtst['name'].str.contains('уковод|иректор|прав', regex=True), True, False)
xtst['oper'] = np.where(xtst['name'].str.contains('пер', regex=True), True, False)
xtst['prod'] = np.where(xtst['name'].str.contains('родав', regex=True), True, False)
xtst['engi'] = np.where(xtst['name'].str.contains('нжене', regex=True), True, False)
xtst['hr'] = np.where(xtst['name'].str.contains('hr|HR|екрут|ecruit', regex=True), True, False)
xtst['cln'] = np.where(xtst['name'].str.contains('борщ|линин', regex=True), True, False)
xtst['law'] = np.where(xtst['name'].str.contains('юр|Юр', regex=True), True, False)
xtst['prep'] = np.where(xtst['name'].str.contains('епод|читель', regex=True), True, False)
xtst['pov'] = np.where(xtst['name'].str.contains('овар|екарь', regex=True), True, False)
xtst['buh'] = np.where(xtst['name'].str.contains('ухгал', regex=True), True, False)

xtst.drop('name', axis=1, inplace=True)

# lets throw in some heavy artillery

# descr_vectorizer =  TfidfVectorizer(
#     sublinear_tf=True,
#     strip_accents='unicode',
#     analyzer='word',
#     token_pattern=r'\w{1,}',
#     ngram_range=(1,1),
#     max_features=100)

# xtrain_raw['description'].fillna(' ', inplace=True),
# xtest_raw['description'].fillna(' ', inplace=True),

# all_descrs = pd.concat([
#      xtrain_raw['description'], 
#      xtest_raw['description']
# ])

# descr_vectorizer.fit(all_descrs)

# xtr_descr = descr_vectorizer.transform(xtrain_raw['description'])
# dftr = pd.DataFrame(xtr_descr.toarray(), columns=descr_vectorizer.get_feature_names_out())
# xtst_descr = descr_vectorizer.transform(xtest_raw['description'])
# dfts = pd.DataFrame(xtst_descr.toarray(), columns=descr_vectorizer.get_feature_names_out())

xtr.drop('description', axis=1, inplace=True)
xtst.drop('description', axis=1, inplace=True)

# xtr = pd.concat([xtr, dftr.set_index(xtr.index)], axis=1)
# xtst = pd.concat([xtst, dfts.set_index(xtst.index)], axis=1)

# xtr.fillna(0.0)
# xtst.fillna(0.0)

xtr.head()

,has_test,response_letter_required,year,month,emp_length,hasremote,hashigh,hassocial,hasfree,hastk,...,ruk,oper,prod,engi,hr,cln,law,prep,pov,buh
id,,,,,,,,,,,,,,,,,,,,,
29083,False,False,2020,6,15,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
26052,False,False,2020,7,13,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
24055,False,False,2020,4,8,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4408,False,False,2020,8,9,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
16856,False,True,2021,9,13,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [69]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer

def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

smape_score = make_scorer(smape, greater_is_better=False)
ytr = ytrain_raw.values.ravel()
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=8, max_features='sqrt',
                                   warm_start=True,
                                   loss='absolute_error', random_state =42)

In [70]:
gbr.fit(xtr,ytr)
smape_score(gbr, xtr, ytr)
xid = pd.read_csv('X_test.csv',usecols=['id'])
xid['salary_to'] = gbr.predict(xtst)
xid.to_csv('submission.csv', index=False)